In [1]:
import pandas as pd

In [2]:
# Data obtained from https://information.stpaul.gov/Public-Safety/Traffic-Stop-Dataset/kkd6-vvns
df = pd.read_csv('./data/stpaul.csv')

In [3]:
df.head()

,YEAR OF STOP,DATE OF STOP,RACE OF DRIVER,GENDER OF DRIVER,DRIVER SEARCHED?,VEHICLE SEARCHED?,CITATION ISSUED?,AGE OF DRIVER,REASON FOR STOP,POLICE GRID NUMBER,LOCATION OF STOP BY POLICE GRID,COUNT
0,2018,05/11/2018 08:47:00 PM,Black,Male,No,No,No,NaN,Moving Violation,991.0,NaN,1
1,2019,03/03/2019 12:15:00 AM,Black,Male,No,No,No,NaN,Equipment Violation,991.0,NaN,1
2,2019,02/13/2019 01:21:00 PM,Black,Male,No,No,Yes,28.0,Moving Violation,991.0,NaN,1
3,2019,07/27/2019 09:23:00 PM,White,Male,No,No,No,NaN,Equipment Violation,991.0,NaN,1
4,2018,01/20/2018 09:19:00 PM,White,Male,No,No,No,NaN,Moving Violation,991.0,NaN,1


In [4]:
drop_cols = [
    'YEAR OF STOP', 
    'RACE OF DRIVER', 
    'GENDER OF DRIVER',
    'DRIVER SEARCHED?', 
    'VEHICLE SEARCHED?', 
    'CITATION ISSUED?',
    'AGE OF DRIVER',
    'LOCATION OF STOP BY POLICE GRID', 
    'COUNT'
]

In [5]:
df.drop(columns=drop_cols, inplace=True)

In [6]:
df['POLICE GRID NUMBER'].isnull().sum()

17

In [7]:
df.dropna(subset=['POLICE GRID NUMBER'], inplace=True)

In [8]:
df["POLICE GRID NUMBER"] = df["POLICE GRID NUMBER"].astype(int)

In [9]:
df = df[ df["REASON FOR STOP"] == "Equipment Violation" ].drop(columns="REASON FOR STOP")

In [10]:
df["DATE OF STOP"] = pd.to_datetime(df["DATE OF STOP"])

In [11]:
df["month"] = df["DATE OF STOP"].dt.month
df["day"] = df["DATE OF STOP"].dt.day

In [12]:
df = df.loc[ (df.month==7) & (df.day>=1) & (df.day<=7) ]

In [13]:
df.head()

,DATE OF STOP,POLICE GRID NUMBER,month,day
676,2018-07-04 17:03:00,71,7,4
742,2018-07-02 00:13:00,91,7,2
933,2019-07-02 22:40:00,56,7,2
1062,2018-07-07 20:59:00,71,7,7
1073,2018-07-07 10:28:00,91,7,7


In [14]:
counts = df.groupby("POLICE GRID NUMBER")["DATE OF STOP"].count()
counts.name = "counts"

In [15]:
counts = pd.DataFrame({'POLICE GRID NUMBER': counts.index, 'incidents': counts.values})

In [16]:
counts.head()

,POLICE GRID NUMBER,incidents
0,6,1
1,9,1
2,11,8
3,12,3
4,13,1


In [17]:
df_grid = pd.read_csv('./data/stpaul-PoliceGrid.csv')

In [18]:
df_grid.head()

,the_geom,DIST,GRIDNUM
0,MULTIPOLYGON (((-93.1873187332505 44.959845223...,Western District,82
1,MULTIPOLYGON (((-93.1891517284332 44.961503652...,Western District,62
2,MULTIPOLYGON (((-93.18726521154463 44.97865486...,Western District,43
3,MULTIPOLYGON (((-93.18723966266045 44.98348311...,Western District,22
4,MULTIPOLYGON (((-93.18741336776411 44.96870629...,Western District,63


In [19]:
df_grid = df_grid[["DIST", "GRIDNUM"]]

In [20]:
counts = counts\
    .merge(df_grid, left_on='POLICE GRID NUMBER', right_on='GRIDNUM')\
    .drop(columns='POLICE GRID NUMBER')\
    .sort_values(by='incidents', ascending=False)

In [21]:
counts.head(10)

,incidents,DIST,GRIDNUM
24,25,Eastern District,54
38,19,Eastern District,74
25,16,Eastern District,55
49,13,Western District,89
50,12,Central District,90
40,12,Eastern District,76
55,12,Eastern District,95
12,11,Eastern District,34
23,9,Eastern District,53
21,9,Central District,50


Use https://information.stpaul.gov/Public-Safety/Saint-Paul-Police-Grid-Shapefile/ykwt-ie3e to find the geographic area:  

Grid Number | Neighborhood
--- | ---
54 | Payne Phalen
74 | Payne Phalen (near Swede Hollow Park)
55 | Payne Phalen
89 | Thomas - Dale, North Frogtown
90 | North Frogtown
76 | Parkway - Greenbrier
95 | North of Dayton's Bluff
34 | Payne Phalen
53 | West of Payne Phalen
50 | "South of Maryland"

In [22]:
ns = pd.DataFrame({
    'GRIDNUM': [54,74,55,89,90,76,95,34,53,50,], 
    'neighborhood': ["Payne Phalen", 
        "Payne Phalen (near Swede Hollow Park)", 
        "Payne Phalen", 
        "Thomas - Dale, North Frogtown", 
        "North Frogtown", 
        "Parkway - Greenbrier", 
        "North of Dayton's Bluff", 
        "Payne Phalen", 
        "West of Payne Phalen", 
        "'South of Maryland'", ]
})

In [23]:
counts.head(10).merge(ns, on='GRIDNUM').drop(columns=['DIST', 'GRIDNUM'])

,incidents,neighborhood
0,25,Payne Phalen
1,19,Payne Phalen (near Swede Hollow Park)
2,16,Payne Phalen
3,13,"Thomas - Dale, North Frogtown"
4,12,North Frogtown
5,12,Parkway - Greenbrier
6,12,North of Dayton's Bluff
7,11,Payne Phalen
8,9,West of Payne Phalen
9,9,'South of Maryland'
